In [2]:
# Unlike Python, Julia doesn't require optional arguments to have a keyword
function fun(x, y=4, z=3im) 
    println("x = $x y = $y z = $z")
end

fun (generic function with 3 methods)

In [4]:
fun(1)

x = 1 y = 4 z = 0 + 3im


In [5]:
fun(2, 100)

x = 2 y = 100 z = 0 + 3im


In [6]:
# We can also assign keywords so that order isn't as important to make using functions with many arguments easier
function f(x; y)
    # y isn't optional but needs to be passed as f(1; y=2)
    println("x = $x y = $y")
end

f (generic function with 1 method)

In [7]:
f(1)

UndefKeywordError: UndefKeywordError: keyword argument y not assigned

In [8]:
f(1, 2)

MethodError: MethodError: no method matching f(::Int64, ::Int64)
Closest candidates are:
  f(::Any; y) at In[6]:4

In [9]:
f(1; 2)

ErrorException: syntax: invalid keyword argument syntax "2"

In [10]:
f(1; y=2)

x = 1 y = 2


In [11]:
# This is pretty weird though. Usually if we name an argument we give it a default. 
function f(x; y=1)
    println("x = $x y = $y")
end

f (generic function with 1 method)

In [13]:
f(1, y=2)

x = 1 y = 2


In [14]:
f(1)

x = 1 y = 1


Here's something cool: the **do-block syntax**. Passing functions as arguments can be awkward if the function spans multiple lines: 

In [15]:
map(x -> begin 
            if x < 0
                return 0 
            elseif x == 0
                return 1 
            else 
                return x
            end
        end, [-1, 0, 1, 2])

4-element Array{Int64,1}:
 0
 1
 1
 2

The keyword `do` makes it a little more clear: 

In [16]:
map([-1, 0, 1, 2]) do x
    if x < 0
        return 0
    elseif x == 0
        return 1 
    else 
        return x
    end
end

4-element Array{Int64,1}:
 0
 1
 1
 2

`do` creates an anonymous function and passes `x` as the argument. We can have multiple argument functions too, and then passes it as the first argument to `map`. *How* the function is used is based on the outer function (`map` here, which applies the function to each item in the array). 

Here's another use that's pretty cool: the `open` function has multiple different versions with different parameters. One allows us to specify a function which is called after opening a file, and then after the function the file is closed. We can use this functionality to avoid writing the code to close a file because that's messier. 

In [21]:
# Definition from source
?open

search: open isopen propertynames CompositeException operm hasproperty



```
open(f::Function, args...; kwargs....)
```

Apply the function `f` to the result of `open(args...; kwargs...)` and close the resulting file descriptor upon completion.

# Examples

```jldoctest
julia> open("myfile.txt", "w") do io
           write(io, "Hello world!")
       end;

julia> open(f->read(f, String), "myfile.txt")
"Hello world!"

julia> rm("myfile.txt")
```

---

```
open(filename::AbstractString; keywords...) -> IOStream
```

Open a file in a mode specified by five boolean keyword arguments:

| Keyword    | Description            | Default                               |
|:---------- |:---------------------- |:------------------------------------- |
| `read`     | open for reading       | `!write`                              |
| `write`    | open for writing       | `truncate \| append`                  |
| `create`   | create if non-existent | `!read & write \| truncate \| append` |
| `truncate` | truncate to zero size  | `!read & write`                       |
| `append`   | seek to end            | `false`                               |

The default when no keywords are passed is to open files for reading only. Returns a stream for accessing the opened file.

---

```
open(filename::AbstractString, [mode::AbstractString]) -> IOStream
```

Alternate syntax for open, where a string-based mode specifier is used instead of the five booleans. The values of `mode` correspond to those from `fopen(3)` or Perl `open`, and are equivalent to setting the following boolean groups:

| Mode | Description                   | Keywords                       |
|:---- |:----------------------------- |:------------------------------ |
| `r`  | read                          | none                           |
| `w`  | write, create, truncate       | `write = true`                 |
| `a`  | write, create, append         | `append = true`                |
| `r+` | read, write                   | `read = true, write = true`    |
| `w+` | read, write, create, truncate | `truncate = true, read = true` |
| `a+` | read, write, create, append   | `append = true, read = true`   |

# Examples

```jldoctest
julia> io = open("myfile.txt", "w");

julia> write(io, "Hello world!");

julia> close(io);

julia> io = open("myfile.txt", "r");

julia> read(io, String)
"Hello world!"

julia> write(io, "This file is read only")
ERROR: ArgumentError: write failed, IOStream is not writeable
[...]

julia> close(io)

julia> io = open("myfile.txt", "a");

julia> write(io, "This stream is not read only")
28

julia> close(io)

julia> rm("myfile.txt")
```

---

```
open(fd::OS_HANDLE) -> IO
```

Take a raw file descriptor wrap it in a Julia-aware IO type, and take ownership of the fd handle. Call `open(Libc.dup(fd))` to avoid the ownership capture of the original handle.

!!! warn
    Do not call this on a handle that's already owned by some other part of the system.


---

```
open(command, mode::AbstractString, stdio=devnull)
```

Run `command` asynchronously. Like `open(command, stdio; read, write)` except specifying the read and write flags via a mode string instead of keyword arguments. Possible mode strings are:

| Mode | Description | Keywords                    |
|:---- |:----------- |:--------------------------- |
| `r`  | read        | none                        |
| `w`  | write       | `write = true`              |
| `r+` | read, write | `read = true, write = true` |
| `w+` | read, write | `read = true, write = true` |

---

```
open(command, stdio=devnull; write::Bool = false, read::Bool = !write)
```

Start running `command` asynchronously, and return a `process::IO` object.  If `read` is true, then reads from the process come from the process's standard output and `stdio` optionally specifies the process's standard input stream.  If `write` is true, then writes go to the process's standard input and `stdio` optionally specifies the process's standard output stream. The process's standard error stream is connected to the current global `stderr`.

---

```
open(f::Function, command, args...; kwargs...)
```

Similar to `open(command, args...; kwargs...)`, but calls `f(stream)` on the resulting process stream, then closes the input stream and waits for the process to complete. Returns the value returned by `f`.


In [23]:
data = "I wrote this with Julia"
open("test.txt", "w") do io
    write(io, data)
end

23

We can combine functions in Julia with 2 methods. 

First is the composition \circ+Tab. 

In [50]:
# Not sure why this doesn't work
(√ ∘ +)(10, 6)

LoadError: syntax: "∘" is not a unary operator

In [51]:
# Asked on Slack and they said it's because of how √ is parsed 
((√) ∘ +)(10, 6)

4.0

In [42]:
√(+(10,6))

4.0

In [58]:
√(4)

2.0

The second method is to pipe (aka chain) functiosn to send data from output to input. 

In [48]:
1:10 |> sum |> sqrt

7.416198487095663

In [49]:
(sqrt ∘ sum)(1:10)

7.416198487095663

The useful thing with piping is that we can broadcast some functions to dot notation for example `.|>`

In [73]:
# This showcases the dot-vectorization notation described next; notice how we apply uppercase to first string, 
# then reverse to second, etc
["a", "list", "of", "strings"] .|> [uppercase, reverse, titlecase, length]

MethodError: MethodError: objects of type Array{Function,1} are not callable
Use square brackets [] for indexing an Array.

Most languages have vectorized versions of functions to avoid writing loops for speed versions. We don't to do this in Julia but they can still be convenient. So **any function** can be applied elementwise with the syntax `f.(A)`. 

In [74]:
sin.([1,2,3,4,5])

5-element Array{Float64,1}:
  0.8414709848078965
  0.9092974268256817
  0.1411200080598672
 -0.7568024953079282
 -0.9589242746631385

Generally, `f.(args...) = broadcast(f, args...)` which means we can operate on multiple arrays: 

In [75]:
f(x, y) = print("$x, $y\n")

f (generic function with 2 methods)

In [79]:
A = [1,2,3,4,5]
f(pi, A)
f.(pi, A)
f.(A, A)

π, [1, 2, 3, 4, 5]
π, 1
π, 2
π, 3
π, 4
π, 5
1, 1
2, 2
3, 3
4, 4
5, 5


5-element Array{Nothing,1}:
 nothing
 nothing
 nothing
 nothing
 nothing

Since adding dots to code is tedious, the macro `@.` converts *every* function call, operation, and assignment in an expression to the "dotted" version

In [86]:
Y = [1.0, 2.0, 3.0, 4.0, 5.0]
X = similar(Y) # pre-allocate output array
@. X = sin(cos(Y)) # Same as X .= sin.(cos.(Y))

5-element Array{Float64,1}:
  0.5143952585235492
 -0.4042391538522658
 -0.8360218615377305
 -0.6080830096407656
  0.2798733507685274

In [88]:
# This was the example above 
[1:2;] .|> [x->2x, inv]

2-element Array{Real,1}:
 2
 0.5

In [89]:
[1:3]

1-element Array{UnitRange{Int64},1}:
 1:3

In [90]:
[1:3;]

3-element Array{Int64,1}:
 1
 2
 3